In [556]:
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx 
from icecream import ic
import random

In [564]:
#CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])


DISTANCE_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DISTANCE_MATRIX[c1.Index, c2.Index] = DISTANCE_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

In [565]:
def solution_cost(solution):
    assert solution[0] == solution[-1]        # start city = end city
    assert set(solution) == set(range(len(CITIES)))        # check if all cities are visited
    
    cost_tot = 0
    for c1, c2 in zip(solution, solution[1:]):
        cost_tot += DISTANCE_MATRIX[c1, c2]
    return cost_tot

## 1 step lookahead (Greedy)
At each step we select a city conseidering the smaller sum of distances between the current city, the new one and the one after the new one.<br>

*IDEA and HELP:* **Gabriele Pirilli** 

In [566]:
it_counter = 0

best_cost = np.inf
best_solution = np.nan


dist_m = DISTANCE_MATRIX.copy()              # it will be modified
visited_cities = np.full(len(CITIES), False)
    
solution = []
start_city = 0
solution.append(start_city)

city = start_city
visited_cities[city] = True
  
    
while not np.all(visited_cities):
    
    dist_m[:, city] = np.inf
    
    not_visited_cities = [x for x in range(len(visited_cities)) if not visited_cities[x]]
    #ic(not_visited_cities)

    best_next_city = None
    best_two_step_cost = np.inf

    # Find the best next city and its two-step cost
    for next_city in not_visited_cities:
        it_counter += 1
        
        dist_m_new = dist_m.copy()  # Copy to modify distances temporarily
        dist_m_new[:, next_city] = np.inf

        #print(lookahead_cities)
        
        if len(not_visited_cities) == 1:         # Handling the last city
            best_next_city = not_visited_cities[0]
            two_step_cost = dist_m[city, next_city] 
            
        else:
            lookahead_cities = not_visited_cities.copy()
            np.delete(lookahead_cities, np.where(lookahead_cities == next_city))    # We remove the current city
            
            closest_lookahead_city = min(lookahead_cities, key=lambda x: dist_m_new[next_city, x])
            two_step_cost = dist_m[city, next_city] + dist_m_new[next_city, closest_lookahead_city]
           
            if two_step_cost < best_two_step_cost:
                best_two_step_cost = two_step_cost
                best_next_city = next_city  
        
    visited_cities[best_next_city] = True       # Move to the best next city
    solution.append(best_next_city)
    city = best_next_city       # Update 

    
solution.append(start_city)    # Closing the circle
cost = solution_cost(solution)

#print("Final solution path:", solution)
print("Final solution cost:", cost)
print("Number of inner iterations", it_counter)

Final solution cost: 1475.5280911045297
Number of inner iterations 28


## Genetic agorithm 

In [567]:
# For reproducibility
seed_value = 23
np.random.seed(seed_value)
random.seed(seed_value)

Starting point = the result of the Greedy algorithm and its variants. <br><br>
The **Hyper-modern GA** was implemented. At each iteration or mutation or crossover was performed. 3 types of mutations (one choosen randomly at each iteration) and 1 type of crossover are available.

In [568]:
seed_value = 23
np.random.seed(seed_value)
random.seed(seed_value)


### FUNCTIONS
class Individual:
    def __init__(self, genome: np.ndarray, fitness: float = None):
        self.genome = genome
        self.fitness = fitness if fitness is not None else self.calculate_fitness()

    def calculate_fitness(self) -> float:
        return -float(solution_cost(self.genome))  
    
    
def fitness(individual):      #genome = solution
    return -float(solution_cost(individual.genome))



### PARENT SELECTIONS:
def random_parent_selection(population, n=5):   ### TOURNAMENT SELECTION ###
    candidates1 = sorted(np.random.choice(population, n), key=lambda e: e.fitness, reverse=True)   # We choose randomly and then sort by fitness
    parent_1 = candidates1[0]   #The best among n selected
    
    candidates2 = sorted(np.random.choice(population, n), key=lambda e: e.fitness, reverse=True)
    parent_2 = candidates2[0]
    return parent_1, parent_2


def fitness_parent_selection(population):   ### FITNESS-PROPORTIONAL SELECTION ###
    total_fitness = sum(ind.fitness for ind in population)      # Total fitness
    selection_probs = [ind.fitness / total_fitness for ind in population]   #Selection prob. proportional to fitness
    selected_individuals = np.random.choice(population, 2, replace=False, p=selection_probs)   # Selecting 2 parents
    return selected_individuals[0], selected_individuals[1]




### CROSSOVERS:
def inver_over_crossover(p1: Individual, p2: Individual):   #Function done with Gabriele Pirilli
    genome1 = p1.genome
    genome2 = p2.genome
    
    child_genome = [None] * len(genome1)
    child_genome[0] = genome1[0]  # Fixed start
    child_genome[-1] = genome1[-1]  # Fixed end

    # Randomly select two crossover points from the middle of the genome
    start_idx = np.random.randint(1, len(genome1) - 1)    # Avoid first and last element
    end_idx = np.random.randint(1, len(genome1) - 1)
    
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx

    child_genome[start_idx:end_idx + 1] = genome1[start_idx:end_idx + 1]

    current_pos = 1  # Start filling after the first fixed position
    for gene in genome2:
        if gene not in child_genome:
            while current_pos < len(child_genome) - 1 and child_genome[current_pos] is not None:
                current_pos += 1
            if current_pos < len(child_genome) - 1:  # Ensure we don't overwrite the last fixed element
                child_genome[current_pos] = gene

    new_individual = Individual(np.array(child_genome))
    return new_individual



### MUTATIONS:
def n_swap_mutation(individual: Individual, n=1):   # Chose 2 positions and swap for n times
    mutated_genome = individual.genome.copy()
    genome_length = len(mutated_genome)
    
    for _ in range(n):        
        idx1, idx2 = np.random.choice(range(1, genome_length-1), 2, replace=False)      # so we mantain first element = last element 
        mutated_genome[idx1], mutated_genome[idx2] = mutated_genome[idx2], mutated_genome[idx1]
        
    new_individual = Individual(mutated_genome)
    return new_individual


def insert_mutation(individual: Individual):   # Select 2 indices, put them near and the elements in between will be after them
    mutated_genome = individual.genome
    genome_length = len(mutated_genome)
    
    idx1, idx2 = np.random.choice(range(1, genome_length-1), 2, replace=False)
    
    if idx1 > idx2:    # idx1 will be for sure the smaller
        idx1, idx2 = idx2, idx1
    
    element = mutated_genome[idx2]
    mutated_genome = np.delete(mutated_genome, idx2)     # Remove idx2 from its original position
    mutated_genome = np.insert(mutated_genome, idx1 + 1, element)    # Insert the element right after idx1
    
    new_individual = Individual(mutated_genome)
    return new_individual



def inversion_mutation(individual: Individual):     # Select 2 idxs and swap them and all the elements in between
    mutated_genome = individual.genome.copy()
    genome_length = len(mutated_genome)
    
    idx1, idx2 = np.random.choice(range(1, genome_length-1), 2, replace=False)
    if idx1 > idx2:      # idx1 will be for sure the first
        idx1, idx2 = idx2, idx1
    mutated_genome[idx1:idx2 + 1] = mutated_genome[idx1:idx2 + 1][::-1]  # Inverting

    new_individual = Individual(mutated_genome)
    return new_individual

##### REMARK: The last element must be always equal to the first one! <br>
So we consider range(1, genome_length-1) avoiding to select the 1st and last element in MUTATIONS.

In [569]:
### PARAMETER DEFINITIOS

print('Number of cities:', len(CITIES))

nnn = 2   #To define how strong swaps to do, how many mutations at each iteration, how big the population should be
div = 2
p=[0.9, 0.1]    # More probability of mutation than recombination
if len(CITIES)>100:
    nnn = 4
    div = 4
    ppp=[0.8, 0.2]  
elif len(CITIES)>300:
    nnn = 6
    div = 6
    ppp=[0.8, 0.2]
elif len(CITIES)>500:
    nnn = 8
    div = 8
    ppp=[0.8, 0.2]
    
POPULATION_SIZE = int(len(CITIES)/div)
MAX_GENERATIONS = 100000

Number of cities: 8


In [570]:
### Initializations

seed_value = 23
np.random.seed(seed_value)
random.seed(seed_value)

n_it_tot = it_counter         # We already searched for a "good" solution
initial_individual = Individual(solution)       # Greedy solution
print('Greedy: ',solution_cost(initial_individual.genome))

population = []
population.append(initial_individual)

mutation_probabilities = [1/3, 1/3, 1/3]
mutations = [n_swap_mutation, insert_mutation, inversion_mutation]
crossover = inver_over_crossover

champion = max(population, key=lambda individual: individual.fitness)    #Best individual so far
best_fitness = champion.fitness
n_it_no_change = 0     # 

for g in range(MAX_GENERATIONS):
    n_it_tot = n_it_tot + 1
    
    if n_it_no_change >= 1000:
        break
    
    
    #First step only mutation => Generating additional individuals to create the initial population
    if g == 0:   
        for it in range(POPULATION_SIZE-1):
            new_individual = n_swap_mutation(initial_individual, n=nnn)
            population.append(new_individual)
            
    else:    
        # Genetic operators => Randomly choose if to do mutation only (0) or recombination only(1)
        choice = np.random.choice([0, 1], p=ppp)   # Higher probability to select MUTATION
            
        if choice == 0:  #Mutation only
            selected_mutation = np.random.choice(mutations, p=mutation_probabilities)  # Choose one mutation considering a probability
            selected_individuals = np.random.choice(population, nnn*2, replace=False)     # Select 2 individuals from the population
                
            for i in selected_individuals:     # Mutate those 2 with the selected method
                o = selected_mutation(i)
                population.append(o)
                
        elif choice == 1:   #Recombination only
            #Parent selection:
            pr1, pr2 = random_parent_selection(population)      # one random parent selection
            pf1, pf2 = fitness_parent_selection(population)     # one fitness-proportional selection
                                             
            o_r = crossover(pr1, pr2)
            population.append(o_r)
            o_f = crossover(pf1, pf2)
            population.append(o_f)

  
        
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]   # Selection of the best individuals
    
    current_best = max(population, key=lambda individual: individual.fitness)
    if current_best.fitness > best_fitness:
        best_fitness = current_best.fitness
        n_it_no_change = 0  # Reset stagnant count on improvement
    else:
        n_it_no_change += 1  # Increment stagnant count if no improvement

    champion = current_best


Greedy:  1475.5280911045297


In [571]:
print('Genetic: ', solution_cost(champion.genome))
print('Total it.:', n_it_tot)

Genetic:  1345.5449564733105
Total it.: 1123
